# Necessary imports

In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import shutil
from tqdm import tqdm
import seaborn as sns

# Load KArSL Labels

In [3]:
karsl_df = pd.read_excel('KARSL-502_Labels.xlsx')
karsl_100 = karsl_df[70:170].reset_index(drop=True)
karsl_100

,SignID,Sign-Arabic,Sign-English
0,71,هيكل عظمي,Skeleton
1,72,جمجة,skull
2,73,عمود فقري,Backbone
3,74,قفص صدري,Chest
4,75,جهاز تنفسي,Respiratory device
...,...,...,...
95,166,يشم,inhale
96,167,يصعد,rise
97,168,ينزل,descend
98,169,يفتح,open


In [4]:
def get_features(signer,SignID,split):
  """ Returns lists corresponding to features of all the videos contained
  in the split folder for the given SignID performed by the signer
  Args:
      signer(int): the signer of interest. Could be 1 or 2 or 3
      SignID(int): the ID of the given gloss
      split(str): can be 'train', 'test'

  Returns:
      videos_SignID(list):the SignIDs of the videos
      videos_signer_id(list): the ids of the signers who performed the signs
      videos_Sign_Arabic(list): the labels of the signs in arabic
      videos_Sign_English(list): the labels of the signs in english
      videos_SequenceID(list): the IDs of each video
      videos_num_frames(list): the number of frames of each video
  """
  videos_SignID = []
  videos_signer_id = [] 
  videos_Sign_Arabic = [] 
  videos_Sign_English = [] 
  videos_SequenceID = []
  videos_num_frames = [] 
  sequences=os.listdir(os.path.join(str(signer),split,str(SignID).zfill(4)))
  sequences=[s for s in sequences if s.endswith('.mp4')]
  for sequence in sequences:
    video_SignID = SignID
    video_signer_id = signer
    video_Sign_Arabic = karsl_100.loc[SignID-71]['Sign-Arabic']
    video_Sign_English = karsl_100.loc[SignID-71]['Sign-English']
    video_SequenceID = str(sequence)[:-4]
    cap = cv2.VideoCapture(os.path.join(str(signer),split,str(SignID).zfill(4),str(sequence)))
    video_num_frames =  int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    videos_SignID.append(video_SignID)
    videos_signer_id.append(video_signer_id) 
    videos_Sign_Arabic.append(video_Sign_Arabic) 
    videos_Sign_English.append(video_Sign_English)
    videos_SequenceID.append(video_SequenceID)
    videos_num_frames.append(video_num_frames) 
  return videos_SignID, videos_signer_id, videos_Sign_Arabic, videos_Sign_English, videos_SequenceID, videos_num_frames

In [5]:
def make_features_array(signer, split):
  """Given the split value and the signer, it returns a corresponding dataframe containing 
  features for each video performed by the given signer in the split data.
  Args:
    signer(int): can be 1, 2 or 3
    split(str): can be 'train', 'test' or 'val'.
  Returns:
    features_df(DataFrame): each row from the dataframe corresponds to a video from the split data.
  """
  features_df = pd.DataFrame(columns=['SignID','signer_id', 'Sign-Arabic', 'Sign-English','SequenceID', 'num_frames'])
  for _, row in tqdm(karsl_100.iterrows()):
      videos_SignID, videos_signer_id, videos_Sign_Arabic, videos_Sign_English, videos_SequenceID, videos_num_frames = get_features(signer,row.SignID,split)
      df = pd.DataFrame(list(zip(videos_SignID, videos_signer_id, videos_Sign_Arabic, videos_Sign_English, videos_SequenceID, videos_num_frames)), columns=features_df.columns)
      features_df = pd.concat([features_df,df], ignore_index=True)
  return features_df

In [6]:
test_features_df = make_features_array(1, 'test')
test_features_df

100it [02:02,  1.22s/it]


,SignID,signer_id,Sign-Arabic,Sign-English,SequenceID,num_frames
0,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_53_03)_c,47
1,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_54_36)_c,44
2,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_54_41)_c,48
3,71,1,هيكل عظمي,Skeleton,03_01_0071_(20_11_16_16_17_05)_c,46
4,71,1,هيكل عظمي,Skeleton,03_01_0071_(20_11_16_16_17_37)_c,46
...,...,...,...,...,...,...
795,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_22)_c,30
796,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_31)_c,38
797,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_34)_c,37
798,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_47)_c,34


In [7]:
train_features_df = make_features_array(1, 'train')
train_features_df

100it [09:21,  5.61s/it]


,SignID,signer_id,Sign-Arabic,Sign-English,SequenceID,num_frames
0,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_52_41)_c,29
1,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_52_44)_c,42
2,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_52_49)_c,44
3,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_52_53)_c,47
4,71,1,هيكل عظمي,Skeleton,03_01_0071_(01_12_16_15_52_58)_c,50
...,...,...,...,...,...,...
4201,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_42)_c,30
4202,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_44)_c,29
4203,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_49)_c,34
4204,170,1,يقفل ( يغلق ),close,03_01_0170_(21_10_17_15_29_54)_c,35


# Threshold for frame sampling

In [8]:
total_test_frames = sum(test_features_df['num_frames'])
total_train_frames = sum(train_features_df['num_frames'])
total_test_videos = len(test_features_df)
total_train_videos = len(train_features_df)
f_avg=round((total_train_frames + total_test_frames) / (total_train_videos + total_test_videos))
f_avg

48

# Dataset Classes analysis

In [9]:
def count_videos_frames(split):
  """Given the split value, it adds 3 columns to karsl_05 for the max,min and average number of frames for each word
  in the split data.
  Args:
      split(str): can be 'train', 'test' or 'val'.
  """
  if split == 'test':
    df=test_features_df
  else:
    df=train_features_df
  max_frames,min_frames,avg_frames= [], [], []
  for _,row in karsl_100.iterrows():
    SignID=row['SignID']
    subset= df[df['SignID']==SignID]
    max_val=max(subset['num_frames'])
    min_val=min(subset['num_frames'])
    avg_val=sum(subset['num_frames'])/len(subset['num_frames'])
    max_frames.append(max_val)
    min_frames.append(min_val)
    avg_frames.append(avg_val)
  karsl_100['max_'+split+'_frames']=max_frames
  karsl_100['min_'+split+'_frames']=min_frames
  karsl_100['avg_'+split+'_frames']=avg_frames

In [13]:
count_videos_frames('train')
count_videos_frames('test')

In [14]:
karsl_100

,SignID,Sign-Arabic,Sign-English,max_train_frames,min_train_frames,avg_train_frames,max_test_frames,min_test_frames,avg_test_frames
0,71,هيكل عظمي,Skeleton,63,29,47.833333,51,44,46.750
1,72,جمجة,skull,69,40,54.238095,59,43,52.375
2,73,عمود فقري,Backbone,75,36,43.880952,53,39,46.000
3,74,قفص صدري,Chest,45,32,38.095238,40,30,36.000
4,75,جهاز تنفسي,Respiratory device,55,40,45.404762,57,36,45.750
...,...,...,...,...,...,...,...,...,...
95,166,يشم,inhale,50,28,37.928571,49,30,38.375
96,167,يصعد,rise,57,33,45.071429,55,37,45.375
97,168,ينزل,descend,72,35,52.714286,66,39,53.000
98,169,يفتح,open,50,28,38.357143,48,29,39.000


In [13]:
print(min(karsl_100['min_train_frames']))

15


In [14]:
print(max(karsl_100['max_train_frames']))

116
